## Example course catalog crawler 

This script serves as a basic tutorial for extracting courses of interest from a university. This is by no means the only (or even best way) to go about this process—so if you come up with a process that works better, feel free to implement! If you're unfamiliar with any of the libraries, the comments below annotate reasoning behind each.

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request #handles urls
import urllib.parse 
import linkGrabber #extracts urls
import json #encodes/decodes json 
import csv 
import requests #downloads a webpage to scrape
from bs4 import BeautifulSoup, NavigableString, Tag #beautifulsoup pulls data from HTML
import nltk #NLP tasks
from nltk import word_tokenize
from nltk.stem import PorterStemmer #removes word endings
stemmer = PorterStemmer()

The first thing we want to do is set up a function for standard preprocessing. It's also useful to list all of the URLs we'll need to send requests to before scraping. We want all courses within a 2 year *academic* calendar (as opposed to an annual calendar). 

In [2]:
#keyword preprocessing
def preprocess(keyword):
    keyword = keyword.lower() #lowercase
    keyword = word_tokenize(keyword) #tokenize
    for word in keyword:
        keyword = stemmer.stem(word) #stem 
    return (keyword)

#course catalog URLs - 2 academic years 
urls = ['https://classes.cornell.edu/search/roster/FA17?q=',
        'https://classes.cornell.edu/search/roster/SP18?q=',
        'https://classes.cornell.edu/search/roster/SU18?q=',
        'https://classes.cornell.edu/search/roster/FA18?q=',
        'https://classes.cornell.edu/search/roster/SP19?q=',
        'https://classes.cornell.edu/search/roster/SU19?q=']

Next, we'll want to import our keyword csv, split our keyword lists, and preprocess them. The way the csv is set up, we'll want to split the words that are indicated as technical (`T`) or normative (`N`) and that we've chosen to include (`Y`). You'll notice that preprocessing is useful for some of our words but not for others. Here, we've chosen to manually alter words that are not usefully preprocessed. In this case, it means replacing instances of words that are stemmed to end in i.

[regex is a bitch here]

In [3]:
#import keywords
keywords = pd.read_csv("keywords.csv")
technical = keywords[(keywords['Technical/Normative']=='T') & (keywords['Include']=='Y')].Keyword
normative = keywords[(keywords['Technical/Normative']=='N') & (keywords['Include']=='Y')].Keyword
normative = [preprocess(i) for i in normative]
technical = [preprocess(i) for i in technical] 

#replace keywords of interest
normative = [w.replace('privaci', 'privac') for w in normative]
normative = [w.replace('democraci', 'democra') for w in normative]
normative = [w.replace('equiti', 'equit') for w in normative]
normative = [w.replace('histori', 'histor') for w in normative]
normative = [w.replace('justice', 'justic') for w in normative]
normative = [w.replace('liberti', 'libert') for w in normative]
normative = [w.replace('philosophi', 'philosoph') for w in normative]
normative = [w.replace('societi', 'societ') for w in normative]
normative = [w.replace('polici', 'polic') for w in normative]

technical = [w.replace('ai', '^ai') for w in technical]
technical = [w.replace('cs', '^cs') for w in technical]
technical = [w.replace('ict', '^ict') for w in technical]
technical = [w.replace('ml', '^ml') for w in technical]
technical = [w.replace('nlp', '^nlp') for w in technical]

print(normative)
print(technical)

['account', 'critic', 'democra', 'discrimin', 'equal', 'equit', 'ethic', 'fair', 'femin', 'gender', 'govern', 'histor', 'inequ', 'justic', 'law', 'legal', 'libert', 'moral', 'norm', 'philosoph', 'polit', 'power', 'privac', 'race', 'religi', 'respons', 'right', 'secur', 'social', 'societ', 'surveil', 'transpar', 'valu', 'polic']
['^ai', 'algorithm', 'analyt', 'intellig', 'automat', 'code', 'comput', '^cs', 'cyber', 'data', 'digit', '^ict', 'inform', 'intelligen', 'internet', 'machin', '^ml', 'process', '^nlp', 'platform', 'program', 'robot', 'softwar', 'system', 'technolog']


The process behind extracting relevant courses works in two steps:
1. First, we want to find and extract all courses that contain any instance of a normative keyword.
2. Then, we want search within these courses to see if it also contains a technical keyword.

We initialize a data frame with columns for all of the course items we want to extract. It probably makes the most sense to standardize these feature names across all university scripts so that they're easier to merge in the final compiled dataset for all universities. Our items of interest are:
* The course title: `title`
* The department and course number: `dept_num`
* The course description: `description`
* The number of credits for the course: `credits`
* The course instructor: `instructor`
* The link to the course syllabus (if applicable): `syllabus`
* The university the course is extracted from: `university`
* The term that the course is offered during (fall, spring, summer / year): `term`
* The keyword that triggered the extraction (this is for auditing purposes): `keyword`

In [4]:
#init dfs
cornell = pd.DataFrame(columns=['title','dept_num','description','credits','instructor',
                                'syllabus','university','term','keyword'])

The loop below executes part 1 of our extraction. It's long and kind of messy (sorry), so feel free to play around with the structure if you'd like. The key tasks here are to extract our items of interest based on our search queries and append them to our data frame.

In [ ]:
#roster search for all urls
for url in urls:
    #loop through all normative words and extract relevant elements 
    for word in normative: 
        url_keyword = url + word #NOTE:this structure will likely be different between rosters!
        response = requests.get(url_keyword)
        soup = BeautifulSoup(response.content, 'lxml')
        print("soup", soup)
        #extract relevant elements 
        titles = [p.get_text() for p in soup.select(".title-coursedescr")] #get full desc
        dept_nums = [p.get_text() for p in soup.select('.title-subjectcode')]
        descs = [p.get_text() for p in soup.select('.course-descr')]
        credit = [p.get_text() for p in soup.select('.credit-val')]
        profs = [p.get_text() for p in soup.select('.instructors span.tooltip-iws')]
        syllabi = [p.get_text() for p in soup.select('.enrlgrp-syllabi')] #TODO: extract syllabi link
        syllabi = [p.replace('Syllabi:\n','') for p in syllabi] 
        uni = ['cornell university']
        term = [btn.get_text().strip() for btn in soup.select('button',{'class':'btn btn-default dropdown-toggle'})[:1]]       
        keyword = [word]
        for a,b,c,d,e,f,g,h,i in zip(titles,dept_nums,descs,credit,profs,syllabi,uni,term,keyword):
            cornell = cornell.append({'title': a, 
                                        'dept_num': b,
                                        'description': c,
                                        'credits': d,
                                        'instructor': e,
                                        'syllabus': f,
                                        'university': g,
                                        'term': h,
                                        'keyword': i}, ignore_index=True)

# cornell

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

soup <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<title>Class Roster  - Fall 2017 - Search Results</title>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" property="og:image"/>
<meta content="/images/project/cuseal_full_red240.png?v=4.3.3.2019042408" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="Class Roster" name="application-name"/>
<meta content="Cornell University, Office of the University Registrar" name="author"/>
<meta content="Cornell University" name="copyright"/>
<link href="/images/favicon.ico?v=4.3.3.2019042408" rel="icon" type="image/x-icon"/>
<link href="/bundles/cusascommon/third-party/bootstrap-3.3.6-dist/css/bootstrap.min.css?v=4.3.3.2019042408" media="all" rel="stylesheet"/>
<link href="/third-party/undohtml.css?v=4.3.3.2019042408" media

Now that we've extracted all courses containing a normative keyword of interest, we need to filter our courses to only return titles that contain a normative AND a technical keyword. This is the case for all words except instances of our preprocessed `privac` and `secur`, for which we want to return all courses, even if they don't contain two keywords. To do this, we'll split the courses into two data frames, apply our respective conditions, and then merge them back together. 

In [185]:
exceptions = cornell.loc[(cornell['keyword']=='privac') | (cornell['keyword'] =='secur')]
exceptions

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword
20,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2017,privac
25,Fixed-Income Securities,AEM 4260,Focuses on fixed-income securities including c...,4,"Bogan, V",none,cornell university,Fall 2017,secur
49,Privacy in the Digital Age,CS 5436,This course introduces students to privacy tec...,3-4,"Nissenbaum, H",none,cornell university,Spring 2018,privac
54,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2018,secur
93,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2018,privac
98,National Security Affairs / Preparation for Ac...,AIRS 4401,This course is designed for college seniors an...,3,"Heath, M",none,cornell university,Fall 2018,secur
123,"Internet Law, Privacy and Security",LAW 6568,"This is a survey course in Internet law, with ...",3,"Grimmelmann, J",1 available,cornell university,Spring 2019,privac
128,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2019,secur


In [187]:
#loop through technical keyword list, extract relevant titles
for word in technical:
    df = cornell[cornell['title'].str.contains(word, flags = re.IGNORECASE)]
    df['keyword2'] = word
    
#join keyword cols
df["keyword"] = df["keyword"].map(str) + "," + df["keyword2"]
df = df.drop(columns="keyword2")

df

/Users/brianavecchione/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/brianavecchione/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,title,dept_num,description,credits,instructor,syllabus,university,term,keyword
38,"Gendering Religion, Science and Technology",AMST 2621,"There are several ""just-so stories"" about scie...",4,"Rock-Singer, C",none,cornell university,Spring 2018,"gender,technolog"


NOTE: the above cell is likely not the best nor most simple way to execute this step! Feel free to take special liberties here. It's probably wise to pick out a few titles that you know should be returned manually, then check to see if the script is working as desired. 

In [188]:
#combine dfs 
cornell = pd.concat([df, exceptions])
cornell

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword
38,"Gendering Religion, Science and Technology",AMST 2621,"There are several ""just-so stories"" about scie...",4,"Rock-Singer, C",none,cornell university,Spring 2018,"gender,technolog"
20,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2017,privac
25,Fixed-Income Securities,AEM 4260,Focuses on fixed-income securities including c...,4,"Bogan, V",none,cornell university,Fall 2017,secur
49,Privacy in the Digital Age,CS 5436,This course introduces students to privacy tec...,3-4,"Nissenbaum, H",none,cornell university,Spring 2018,privac
54,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2018,secur
93,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2018,privac
98,National Security Affairs / Preparation for Ac...,AIRS 4401,This course is designed for college seniors an...,3,"Heath, M",none,cornell university,Fall 2018,secur
123,"Internet Law, Privacy and Security",LAW 6568,"This is a survey course in Internet law, with ...",3,"Grimmelmann, J",1 available,cornell university,Spring 2019,privac
128,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2019,secur


Lastly, we want to export our csv. Ideally, all csv files should be written to the courses directory in our repository. 

In [189]:
#export as csv
cornell.to_csv('cornell.csv')